In [1]:
import os
from pathlib import Path
import random
import numpy as np

import torch
from torch.utils.data import DataLoader
import torchmetrics

from eeg2fmri_datasets import EEG2fMRIDataset
from Spectrogram2fMRI import Spectrogram2fMRI

import utils
import data_cfg

from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from matplotlib import animation, rc
rc('animation', html='jshtml')

In [13]:
def create_animation(gt_data: np.ndarray, pred_data: np.ndarray, figsize: int=5):
    """Visual animation for per-slice comparison
    """
    fig, axs = plt.subplots(1, 2, figsize=(figsize*2, figsize))

    gt_slice = utils.normalize_data(gt_data[:, :, 0])
    pred_slice = utils.normalize_data(pred_data[:, :, 0])
    
    im1 = axs[0].imshow(np.rot90(gt_slice, k=-1), cmap='hot')
    im2 = axs[1].imshow(np.rot90(pred_slice, k=-1), cmap='hot')

    axs[0].set_title("Ground-truth")
    axs[1].set_title("Generated")

    def animate_func(i):
        gt_slice = utils.normalize_data(gt_data[:, :, i])
        pred_slice = utils.normalize_data(pred_data[:, :, i])
        
        im1.set_data(np.rot90(gt_slice, k=-1))
        im2.set_data(np.rot90(pred_slice, k=-1))
        
    plt.tight_layout()
    plt.colorbar(im1, ax=axs[0])
    plt.colorbar(im2, ax=axs[1])

    plt.close()
    
    num_slices = gt_data.shape[-1]
    
    return animation.FuncAnimation(fig, animate_func, frames=num_slices, interval=100)

### Load data

In [3]:
data_name = 'CNEPFL'
fmri_channel = 30

assert data_name in ['NODDI', 'Oddball', 'CNEPFL']

data_root = Path(data_cfg.processed_data_roots[data_name])

# NODDI
if data_name == 'NODDI':
    test_list = ['44']

    # global min-max value
    eeg_min = -3.904906883760493
    eeg_max = 7.937204954155734
    
# Oddball
elif data_name == 'Oddball':
    test_ID = [16] # test [9 - 10]
    test_list = []
    for idx in test_ID:
        indv_data = f"sub{idx:03}/task001_run001"
        test_list.append(indv_data)
    
    fmri_channel = 32
    
    # global min-max value
    eeg_min = -2.466110737041575
    eeg_max = 6.480417369333849
    
# CNEPFL
elif data_name == 'CNEPFL':
    individuals = sorted([Path(x).stem for x in os.listdir(data_root)])
    test_list = individuals[-4:]
    test_list = [f"{test}/{test}_run-001" for test in test_list]

    # global min-max value
    eeg_min = -4.551622643288133
    eeg_max = 7.93715188090758

eeg_test, fmri_test = utils.load_h5_from_list(data_root, test_list)

# normalize data (fmri_test is already in range [0 - 1])
eeg_test = utils.normalize_data(eeg_test, base_range=(eeg_min, eeg_max))

  0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
# create datasets
test_dataset = EEG2fMRIDataset(eeg_test, fmri_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

### Load model

In [5]:
# define networks
model =  Spectrogram2fMRI(in_dim=20,hidden_dim=256,out_dim=fmri_channel)


In [6]:
# replace the path to your checkpoint
ckpt_path = 'run/CNEPFL/Spectrogram2fMRI_CNEPFL_250410_114436/epoch_28_best_SSIM_0.78786.pth'
device = "cuda:0"

model.load_state_dict(torch.load(ckpt_path, weights_only=True))
model = model.to(device)
model = model.eval()

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


### Evaluation

In [7]:
ssim_metric = torchmetrics.image.StructuralSimilarityIndexMeasure(data_range=1.0)
psnr_metric = torchmetrics.image.PeakSignalNoiseRatio()

ssim_metric = ssim_metric.to(device)
psnr_metric = psnr_metric.to(device)

total_batch = len(test_loader)

test_ssim_score = 0.0
test_psnr_score = 0.0

for batch in tqdm(test_loader):
    eeg_batch, fmri_batch = batch
    
    eeg_batch = eeg_batch.to(device)
    fmri_batch = fmri_batch.to(device)
    
    # model prediction
    with torch.no_grad():
        pred_fmri = model(eeg_batch)

    # calculate SSIM & PSNR & RMSE
    test_ssim_score += ssim_metric(pred_fmri, fmri_batch).item()
    test_psnr_score += psnr_metric(pred_fmri, fmri_batch).item()

ssim_score = test_ssim_score/total_batch
psnr_score = test_psnr_score/total_batch

print(f"SSIM: {ssim_score:.7f}")
print(f"PSNR: {psnr_score:.7f}")

  0%|          | 0/1376 [00:00<?, ?it/s]

SSIM: 0.7878567
PSNR: 23.8366451


### Predict and visualize results

In [8]:
# randomly sample data
eeg_data, fmri_data = random.choice(test_dataset)
eeg_data = eeg_data.unsqueeze(0)
eeg_data = eeg_data.to(device)

# model prediction
with torch.no_grad():
    pred_fmri = model(eeg_data)

In [14]:
# visualization
pred_data = pred_fmri[0].cpu().numpy()

pred_data = np.transpose(pred_data, (1, 2, 0))
gt_data = np.transpose(fmri_data, (1, 2, 0))

np.save("pre.npy",pred_data)
np.save("gt.npy",gt_data)

create_animation(gt_data, pred_data)